# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

### Предобработка данных

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor, Pool
import lightgbm 
%matplotlib inline

Откроем предоставленные данные и получим по ним информацию 

In [2]:
autos = pd.read_csv('/datasets/autos.csv')
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

В предоставленных данных 15 столбцов и 354369 строк. В 5 столбцах присутствуют пропуски. Все 5 столбцов содержат категорийные признаки, поэтому восстановление данных не представляется возможным. Удалять строки с пропущенными данными считаю некорректным, поскольку потеряется больше 10% данных. Поэтому заполним прпуски в 5 столбцах, используя моду

In [3]:
autos['VehicleType'] = autos['VehicleType'].fillna(autos['VehicleType'].mode().values[0])
autos['Gearbox'] = autos['Gearbox'].fillna(autos['Gearbox'].mode().values[0])
autos['Model'] = autos['Model'].fillna(autos['Model'].mode().values[0])
autos['FuelType'] = autos['FuelType'].fillna(autos['FuelType'].mode().values[0])
autos['NotRepaired'] = autos['NotRepaired'].fillna(autos['NotRepaired'].mode().values[0])

In [4]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        354369 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            354369 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              354369 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           354369 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        354369 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Посмотрим как обстоят дела с числовыми данными

In [5]:
autos.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Так как Price для нашего проекта это target, заполнение пропусков считаю исказит полученный результат, поэтому удалим строки с пропущенными данными. RegistrationYear не должен быть ниже 1950 и выше 2016. Power я думаю меньше 50 и выше 1000 не должно быть. В столбце Kilometer все в пределах нормы. RegistrationMonth нулевые значения поменяем на 1. Остальное оставим как есть

In [6]:
autos = autos.drop(autos.query('Price < 100').index)

0 в столбце RegistrationMonth заменим на 1 в одну строку

In [7]:
autos.loc[autos['RegistrationMonth'] == 0, 'RegistrationMonth'] = 1

Для тех строк, где есть условия, напишем функцию

In [8]:
def new_reg_year(value):
    if value > 2016:
        return 2016
    elif value < 1950:
        return 1950
    else:
        return value
autos['RegistrationYear'] = autos['RegistrationYear'].apply(new_reg_year)

In [9]:
def horse_power(value):
    if value < 50:
        return 50
    elif value > 1000:
        return 1000
    else:
        return value
autos['Power'] = autos['Power'].apply(horse_power)

Посмотрим на результат

In [10]:
autos.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,341055.000000,341055.000000,341055.000000,341055.00000,341055.000000,341055.0,341055.000000
mean,4588.861052,2003.161487,114.078647,128497.76429,5.891627,0.0,50695.796983
std,4514.844700,7.176948,61.365972,37243.80161,3.552481,0.0,25733.510931
min,100.000000,1950.000000,50.000000,5000.00000,1.000000,0.0,1067.000000
25%,1200.000000,1999.000000,70.000000,125000.00000,3.000000,0.0,30453.000000
50%,2900.000000,2003.000000,105.000000,150000.00000,6.000000,0.0,49525.000000
75%,6500.000000,2008.000000,143.000000,150000.00000,9.000000,0.0,71229.000000
max,20000.000000,2016.000000,1000.000000,150000.00000,12.000000,0.0,99998.000000


Выбросы и нулевые значения были исправлены. Проверим на наличие дубликатов

In [11]:
autos.duplicated().sum()

5

Дубликатов по отношению к общему объему инфомации ничтожно мало, поэтому просто удалим их

In [12]:
autos = autos.drop_duplicates().reset_index(drop=True)

В предоставленном датасете слишком много столбцов. Удалим ненужные

In [13]:
autos = autos.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)
autos.columns

Index(['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand', 'NotRepaired'],
      dtype='object')

In [14]:
autos.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,sedan,1993,manual,50,golf,150000,1,petrol,volkswagen,no
1,18300,coupe,2011,manual,190,golf,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


Выделим из всего датасета столбцы с категориальными и числовыми значениями 

In [15]:
numeric_features = ['DateCreated', 'Price', 'RegistrationYear', 'Power', 'Kilometer', 'LastSeen']
categorical_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired', 'Brand']


Сделаем копию нашего датасета и в одной из копий переведем категорийные данные в номинальные

In [16]:
autos_nom = autos.copy()
autos_nom = pd.get_dummies(autos_nom)
autos_nom

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
0,480,1993,50,150000,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1,18300,2011,190,125000,5,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1500,2001,75,150000,6,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,3600,2008,69,90000,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341045,1150,2000,50,150000,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
341046,2200,2005,50,20000,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
341047,1199,2000,101,125000,3,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
341048,9200,1996,102,150000,3,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


### Подготовка к обучению

Выделим признаки и целевой признак

In [17]:
features = autos.drop('Price', axis=1)
target = autos['Price']

Выделим признаки и целевой признак с переведенными категорийными данными

In [18]:
features_nom = autos_nom.drop('Price', axis=1)
target_nom = autos_nom['Price']

Разделим датасет на обучающую, валидационную и тестовые выборки с переведенными категорийными данными

In [19]:
features_train_nom, features_test_nom, target_train_nom, target_test_nom = train_test_split(features_nom, target_nom, test_size=0.4, random_state=12345)
f"Размер обучающей выборки с переведенными категорийными данными: {features_train_nom.shape}"

'Размер обучающей выборки с переведенными категорийными данными: (204630, 313)'

In [20]:
features_test_nom, features_valid_nom, target_test_nom, target_valid_nom = train_test_split(features_test_nom, target_test_nom, test_size=0.5, random_state=12345)
f"Размер обучающей выборки с переведенными категорийными данными: {features_train_nom.shape}"

'Размер обучающей выборки с переведенными категорийными данными: (204630, 313)'

In [21]:
f"Размер тестовой выборки с переведенными категорийными данными: {features_test_nom.shape}"

'Размер тестовой выборки с переведенными категорийными данными: (68210, 313)'

Разделим исходные данные на обучающую, валидационную и тестовую выборки.
Сначала разделим исходный датасет на обучающую и валидационную выборки в пропорции 60% и 40%. Затем разделим валидационную выборку пополам на валидационную и тестовую. 

In [22]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)


In [23]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)
f"Размер обучающей выборки: {features_train.shape}"

'Размер обучающей выборки: (255787, 10)'

In [24]:
f"Размер тестовой выборки: {features_test.shape}"

'Размер тестовой выборки: (85263, 10)'

**Вывод**

Были загружены необходимые для работы библиотеки. Далее был открыт предоставленный датасет и изучен на предмет аномалий в данных, пропусков и дубликатов. В результате были проделаны следующие действия:

- в столбцах с категорийными данными строки с пропусками были заполнены, используя моду;
- в столбце RegistrationMonth нулевые значения были заменены на 1;
- в столбце Price нулевые значения были удалены, чтобы не исказить полученный результат, так как Price это цель проекта;
- в столбце RegistrationYear минимальное значение сделали 1950, а максимальное 2021;
- в столбце Power минимальное значение сделали 50, а максимальное 1000;
- датасет был проверен на наличие дубликатов;
- из-за малого количества дубликаты были удалены;
- были удалены не представляющие ценности данные, а именно столбцы DateCrawled, DateCreated, NumberOfPictures, PostalCode, LastSeen;
- Датасет был разделен на обучающую и тестовую выборки с переведенными категорийными данными и без.

Датасет готов к дальнейшей работе

## Обучение моделей

Обучение будет происходить используя различные модели.

### Линейная регрессия

In [25]:
%%time
LR = LinearRegression()
mse = cross_val_score(LR, features_train_nom, target_train_nom, cv=5, scoring='neg_mean_squared_error')

CPU times: user 42.6 s, sys: 19.1 s, total: 1min 1s
Wall time: 1min 1s


In [26]:
%%time
result = round((-mse.mean()) ** 0.5, 2)
print("RMSE модели линейной регрессии на валидационной выборке:", result)

RMSE модели линейной регрессии на валидационной выборке: 2807.09
CPU times: user 389 µs, sys: 166 µs, total: 555 µs
Wall time: 189 µs


In [27]:
%%time
LR=LinearRegression()
LR.fit(features_train_nom, target_train_nom)

CPU times: user 10.6 s, sys: 4.8 s, total: 15.4 s
Wall time: 15.5 s


LinearRegression()

In [28]:
%%time
predicted_valid = LR.predict(features_valid_nom)

CPU times: user 108 ms, sys: 90.3 ms, total: 198 ms
Wall time: 194 ms


###  LightGBM

In [29]:
%%time

autos_train = lightgbm.Dataset(features_train_nom, label=target_train_nom, free_raw_data=False, categorical_feature=categorical_features)
autos_test = lightgbm.Dataset(features_test_nom, label=target_test_nom)
param_grid = {'learning_rate':[0.1, 0.3, 0.5, 0.7],
             'max_depth':[15,20,25,30],
             }

LGB = lightgbm.LGBMRegressor(n_jobs=7)

grid_search = GridSearchCV(estimator=LGB,
                          param_grid=param_grid,
                          cv=3,
                          n_jobs=-1,
                          verbose=0,
                          scoring='neg_mean_squared_error',
                          )



CPU times: user 651 µs, sys: 284 µs, total: 935 µs
Wall time: 121 µs


In [30]:
%%time
grid_search.fit(features_train_nom, target_train_nom)

CPU times: user 3min 46s, sys: 9.59 s, total: 3min 55s
Wall time: 3min 56s


GridSearchCV(cv=3, estimator=LGBMRegressor(n_jobs=7), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5, 0.7],
                         'max_depth': [15, 20, 25, 30]},
             scoring='neg_mean_squared_error')

In [31]:
grid_search.best_params_

{'learning_rate': 0.5, 'max_depth': 15}

In [32]:
'RMSE для LightGBM модели на валидационной выборке: {:2f}'.format((-grid_search.best_score_)**0.5)

'RMSE для LightGBM модели на валидационной выборке: 1701.976437'

In [33]:
%%time
predictions_LGBM = grid_search.best_estimator_.predict(features_valid_nom)

CPU times: user 553 ms, sys: 66.8 ms, total: 619 ms
Wall time: 679 ms


### RandomForestRegressor

Для модели RandomForestRegressor преобразуем категорийные данные в числовые используя Ordinal Encoder

In [34]:
encoder = OrdinalEncoder()
autos_ordinal = autos.copy()
autos_ordinal[categorical_features] = pd.DataFrame(encoder.fit_transform(autos[categorical_features]),
                            columns=categorical_features)

features_ord = autos_ordinal.drop('Price', axis=1)
target_ord = autos_ordinal['Price']

In [35]:
features_train_ord, features_test_ord, target_train_ord, target_test_ord = train_test_split(features_ord, target_ord, test_size=0.4, random_state=12345)

features_test_ord, features_valid_ord, target_test_ord, target_valid_ord = train_test_split(features_test_ord, target_test_ord, test_size=0.5, random_state=12345)


In [36]:
%time
model = RandomForestRegressor()
model.fit(features_train_ord, target_train_ord)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


RandomForestRegressor()

In [37]:
%%time
predictions_valid = model.predict(features_valid_ord)

CPU times: user 2.88 s, sys: 6.37 ms, total: 2.88 s
Wall time: 2.89 s


In [38]:
%%time
result = (mean_squared_error(target_valid_ord, predictions_valid))**0.5
print('RMSE для модели RandomForestRegressor на валидационной выборке:', result)

RMSE для модели RandomForestRegressor на валидационной выборке: 1670.6113999397764
CPU times: user 2.72 ms, sys: 0 ns, total: 2.72 ms
Wall time: 1.54 ms


## Анализ моделей

Как мы уже писали в начале проекта заказчику важны 3 критерия:

- качество предсказания;
- скорость предсказания;
- время обучения.

Мы подобрали гиперпараметры и обучили 3 модели и на основании проделанной работы можем подвести следующий итог:

- среднееквадратическая ошибка модели Линейной регрессии составила 2807.09, время предказания  составило 222 ms, обучился метод за 19.2 s;
- среднееквадратическая ошибка модели LightGBM составила 1701,97, время предказания  составило 587 ms, обучился метод за 14min 51s;
- среднееквадратическая ошибка модели RandomForestRegressor составила 1671.99, время предказания  составило 2.88 s, обучился метод за 5.96 µs.

Таким образом мы делаем вывод, что из 3 выбранных нами моделей лучший результат у модели RandomForestRegressor

Проверим качество предсказания, скорость предсказания и время обучения модели RandomForestRegressor на тестовой выборке 

In [39]:
model = RandomForestRegressor()
model.fit(features_train_ord, target_train_ord)
predictions_test = model.predict(features_test_ord)
result = (mean_squared_error(target_test_ord, predictions_test))**0.5
print('RMSE для модели RandomForestRegressor на тестовой выборке:', result)

RMSE для модели RandomForestRegressor на тестовой выборке: 1671.0393954594597


Как мы можем видеть разница в результате проверки качества предсказания на разных выборках практически отсутствует!